In [30]:
import pandas as pd
import numpy as np
from scipy import stats
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
from functools import partial

# Data Handling
This notebook slices the data into training data and testing data

## Data
One single dataset from Bloomberg. 
- Daily closing price of BTC future and Daily Closing value of brr. 
- Data is merged by Bloomberg using the compare function


In [31]:
future_brr = pd.read_csv('../data/cleaned_data/BBT_Tiingo.csv')

In [33]:
future_brr = future_brr.iloc[:,1:]

In [46]:
future_brr.tail()

,Date,PX_LAST,contract_name,BTC Price,log return future,log return bitcoin
863,2017-12-22 21:00:00+00:00,14135.0,BTCF18 Curncy,14181.751265,-0.081158,-0.071800
864,2017-12-21 21:00:00+00:00,15330.0,BTCF18 Curncy,15237.453003,-0.105752,-0.090475
865,2017-12-20 21:00:00+00:00,17040.0,BTCF18 Curncy,16680.348192,-0.065858,-0.055488
866,2017-12-19 21:00:00+00:00,18200.0,BTCF18 Curncy,17632.061828,-0.048267,-0.058609
867,2017-12-18 21:00:00+00:00,19100.0,BTCF18 Curncy,18696.348689,-0.020726,0.052421


## Slicing data

In [36]:
train_size = 300
test_size = 5
step_size = 5

In [37]:
len(future_brr)

868

In [39]:
data_name = 'BBT_Tiingo'

In [40]:
# n = int(np.floor((len(future_brr)-train_size)/step_size))
n = int(np.floor((len(future_brr) - train_size - test_size)/step_size))
train_set = []
test_set = []

for i in range(n):
    data = future_brr.iloc[(i*step_size):((i)*step_size)+train_size+test_size]
    test = data.iloc[:test_size,]
    train = data.iloc[test_size:,]
    train_set.append(train)
    test_set.append(test)

for i in range(len(train_set)):
    train_set[i].to_csv("../processed_data/%s/train/" %data_name +str(i)+".csv")
    test_set[i].to_csv("../processed_data/%s/test/" %data_name +str(i)+".csv")

In [41]:
test_set[0].head()

,Date,PX_LAST,contract_name,BTC Price,log return future,log return bitcoin
0,2021-05-27 20:00:00+00:00,38865.0,BTCM1 Curncy,38786.472240,0.006583,0.007532
1,2021-05-26 20:00:00+00:00,38610.0,BTCM1 Curncy,38495.443757,0.032643,0.031101
2,2021-05-25 20:00:00+00:00,37370.0,BTCM1 Curncy,37316.628068,-0.062245,-0.060677
3,2021-05-24 20:00:00+00:00,39770.0,BTCM1 Curncy,39650.983644,0.094330,0.093435
4,2021-05-21 20:00:00+00:00,36190.0,BTCM1 Curncy,36114.011225,-0.108809,-0.105785


In [42]:
train_set[0].head()

,Date,PX_LAST,contract_name,BTC Price,log return future,log return bitcoin
5,2021-05-20 20:00:00+00:00,40350.0,BTCM1 Curncy,40143.729949,0.021291,0.018658
6,2021-05-19 20:00:00+00:00,39500.0,BTCM1 Curncy,39401.672539,-0.090465,-0.093476
7,2021-05-18 20:00:00+00:00,43240.0,BTCM1 Curncy,43262.433661,-0.019694,-0.021612
8,2021-05-17 20:00:00+00:00,44100.0,BTCM1 Curncy,44207.612742,-0.131645,-0.126287
9,2021-05-14 20:00:00+00:00,50305.0,BTCM1 Curncy,50158.276955,0.036850,0.032827


In [43]:
train_set[0].tail()

,Date,PX_LAST,contract_name,BTC Price,log return future,log return bitcoin
300,2020-03-20 20:00:00+00:00,6230.0,BTCJ20 Curncy,6194.401518,-0.006400,-0.003845
301,2020-03-19 20:00:00+00:00,6270.0,BTCJ20 Curncy,6218.262575,0.159615,0.153577
302,2020-03-18 20:00:00+00:00,5345.0,BTCJ20 Curncy,5332.998328,-0.012088,-0.009687
303,2020-03-17 20:00:00+00:00,5410.0,BTCJ20 Curncy,5384.912732,0.089872,0.084526
304,2020-03-16 20:00:00+00:00,4945.0,BTCJ20 Curncy,4948.455895,-0.083382,-0.089169


In [44]:
test_set[1].head()

,Date,PX_LAST,contract_name,BTC Price,log return future,log return bitcoin
5,2021-05-20 20:00:00+00:00,40350.0,BTCM1 Curncy,40143.729949,0.021291,0.018658
6,2021-05-19 20:00:00+00:00,39500.0,BTCM1 Curncy,39401.672539,-0.090465,-0.093476
7,2021-05-18 20:00:00+00:00,43240.0,BTCM1 Curncy,43262.433661,-0.019694,-0.021612
8,2021-05-17 20:00:00+00:00,44100.0,BTCM1 Curncy,44207.612742,-0.131645,-0.126287
9,2021-05-14 20:00:00+00:00,50305.0,BTCM1 Curncy,50158.276955,0.036850,0.032827


In [45]:
train_set[-1].tail()

,Date,PX_LAST,contract_name,BTC Price,log return future,log return bitcoin
855,2018-01-05 21:00:00+00:00,16580.0,BTCG18 Curncy,16595.352695,0.103151,0.110616
856,2018-01-04 21:00:00+00:00,14955.0,BTCG18 Curncy,14857.531803,-0.005003,-0.007953
857,2018-01-03 21:00:00+00:00,15030.0,BTCG18 Curncy,14976.171419,0.006341,0.008390
858,2018-01-02 21:00:00+00:00,14935.0,BTCG18 Curncy,14851.042103,0.030249,0.033781
859,2017-12-29 21:00:00+00:00,14490.0,BTCG18 Curncy,14357.744828,0.052784,0.038173
